# Crypto Investments Analysis Using K-means and PCA


In [89]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

This section prepares the data before running the K-Means algorithm.

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

In [6]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data

This section uses the elbow method to find the best value for `k`.


In [8]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1, 11))

In [9]:
# Create an empy list to store the inertia values
inertia = []

In [11]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)



In [80]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_original = {
    "k": k,
    "inertia": inertia
}


# Create a DataFrame with the data to plot the Elbow curve
df_elbow_data_original = pd.DataFrame(elbow_data_original)

In [81]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow_data_original.hvplot.line(
    x="k", 
    y= "inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer:** # The best value for k appears to be 4.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

This section uses the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.


In [14]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)

In [15]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)

KMeans(n_clusters=4)

In [16]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_clusters = model.predict(df_market_data_scaled)

# View the resulting array of cluster values.
print(crypto_clusters)

[0 0 1 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 2 0 1 1 3
 1 1 1 1]


In [ ]:
# Create a copy of the DataFrame
df_market_data_scaled_copy = df_market_data_scaled

In [17]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_scaled["CryptoCluster"] = crypto_clusters

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,CryptoCluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,0
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,0
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,1
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,1
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,0


In [82]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_market_data_scaled.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="CryptoCluster",
    hover_cols = ["coin_id"], 
    title = "Scatter Plot by Crypto - k=4"
)

:NdOverlay   [CryptoCluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis

This section performs a principal component analysis (PCA) and reduce the features to three principal components.


In [83]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [84]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_pca_data = pca.fit_transform(df_market_data_scaled)

# View the first five rows of the DataFrame. 
market_pca_data

array([[-8.22587018e-01,  8.32705550e-01,  5.66553788e-01],
       [-7.09903468e-01,  4.54212287e-01,  1.05640205e+00],
       [-3.07899872e-01, -1.84609191e-01, -7.40819732e-01],
       [-3.53704002e-01, -2.39814878e-01, -5.96140944e-01],
       [-1.48496628e+00,  2.01848451e+00,  1.78701064e+00],
       [-7.81024220e-01,  1.38160920e+00,  9.02061784e-01],
       [-8.33237969e-01,  5.19208426e-01,  2.83965275e+00],
       [-6.44167029e-01,  7.29885318e-01,  1.56514267e+00],
       [-8.68641860e-01,  4.21293449e-01,  6.94078511e-01],
       [-6.49171841e-01, -2.25935254e-01, -3.92028832e-01],
       [-4.07121417e-02, -1.38929203e+00, -1.47935305e+00],
       [-3.11314371e-01, -1.92037612e-01, -7.62095210e-01],
       [-5.56771239e-01, -4.97469252e-01, -6.74255080e-01],
       [-3.10517524e-01,  2.92072674e+00,  1.62486583e+00],
       [-3.14660050e-01, -1.21672792e-01, -1.41549135e-01],
       [-1.01969469e+00, -5.07677556e-01,  1.13693664e+00],
       [ 2.29682729e-01, -1.27264554e+00

In [85]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.3738947 , 0.32920008, 0.1850154 ])

#### Answer the following question: What is the total explained variance of the three principal components?

**Question:** What is the total explained variance of the three principal components?

**Answer:** # The total explained variance of the three principal components is approimately 88%.

In [86]:
# Create a new DataFrame with the PCA data.
df_market_data_pca = pd.DataFrame(market_pca_data, columns=["PC1", "PC2", "PC3"])

# Copy the crypto names from the original data
df_market_data_pca["Crypto"] = df_market_data.index

# Set the coinid column as index
df_market_data_pca = df_market_data_pca.set_index("Crypto")

# Display sample data
df_market_data_pca.head()

,PC1,PC2,PC3
Crypto,,,
bitcoin,-0.822587,0.832706,0.566554
ethereum,-0.709903,0.454212,1.056402
tether,-0.307900,-0.184609,-0.740820
ripple,-0.353704,-0.239815,-0.596141
bitcoin-cash,-1.484966,2.018485,1.787011


---

### Find the Best Value for k Using the PCA Data

This section uses the elbow method to find the best value for `k` using the PCA data.

In [41]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1, 11))

In [42]:
# Create an empy list to store the inertia values
inertia = []

In [43]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_data_pca)
    inertia.append(model.inertia_)
    

In [87]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_data_pca = pd.DataFrame(elbow_data_pca)

In [88]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow_data_pca.hvplot.line(
    x="k", 
    y= "inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** # The best value for k appears to be 4, based on the elbow curve plot.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** The original value was 3.

---

### Cluster Cryptocurrencies with K-means Using the PCA Data

This section uses the PCA data and the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the principal components.


In [52]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)

In [53]:
# Fit the K-Means model using the PCA data
model.fit(df_market_data_pca)

KMeans(n_clusters=4)

In [54]:
# Predict the clusters to group the cryptocurrencies using the PCA data
market_clusters = model.predict(df_market_data_pca)

# View the resulting array of cluster values.
print(market_clusters)

[0 0 3 3 0 0 0 0 0 3 3 3 3 0 3 0 3 3 0 3 3 0 3 3 3 3 3 3 0 3 3 3 1 0 3 3 2
 3 3 3 3]


In [55]:
# Create a copy of the DataFrame with the PCA data
df_market_pca_predictions = df_market_data_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_market_pca_predictions["CryptoCluster"] = market_clusters

# Display sample data
print(df_market_pca_predictions)

                           PC1       PC2       PC3  CryptoCluster
Crypto                                                           
bitcoin              -0.822587  0.832706  0.566554              0
ethereum             -0.709903  0.454212  1.056402              0
tether               -0.307900 -0.184609 -0.740820              3
ripple               -0.353704 -0.239815 -0.596141              3
bitcoin-cash         -1.484966  2.018485  1.787011              0
binancecoin          -0.781024  1.381609  0.902062              0
chainlink            -0.833238  0.519208  2.839653              0
cardano              -0.644167  0.729885  1.565143              0
litecoin             -0.868642  0.421293  0.694079              0
bitcoin-cash-sv      -0.649172 -0.225935 -0.392029              3
crypto-com-chain     -0.040712 -1.389292 -1.479353              3
usd-coin             -0.311314 -0.192038 -0.762095              3
eos                  -0.556771 -0.497469 -0.674255              3
monero    

In [56]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_market_pca_predictions.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="CryptoCluster",
    title = "Scatter Plot by Crypto Asset - PCA=2"
)

:NdOverlay   [CryptoCluster]
   :Scatter   [PC1]   (PC2)

---

### Visualize and Compare the Results

This section visually analyzes the cluster analysis results by contrasting the outcome with and without using the optimization techniques.


In [75]:
# Composite plot to contrast the Elbow curves
original_elbow_chart = df_elbow_data_original.hvplot.line(
    x="k", 
    y= "inertia", 
    title="Elbow Curve", 
    xticks=k
)

pca_elblow_chart = df_elbow_data_pca.hvplot.line(
    x="k", 
    y= "inertia", 
    title="Elbow Curve", 
    xticks=k
)

display(original_elbow_chart + pca_elblow_chart)

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [70]:
# Compoosite plot to contrast the clusters
original_chart = df_market_data_pca.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Crypto",
    title = "Scatter Plot by Crypto Asset - PCA=2 - ORIGINAL"
)

optimal_chart = df_market_pca_predictions.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="CryptoCluster",
    title = "Scatter Plot by Crypto Asset - PCA=2 - PCA"
)

display(original_chart + optimal_chart)

:Layout
   .NdOverlay.I  :NdOverlay   [Crypto]
      :Scatter   [PC1]   (PC2)
   .NdOverlay.II :NdOverlay   [CryptoCluster]
      :Scatter   [PC1]   (PC2)

#### Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** # The scatter chart more appropriately displays the clustered crypto assets after using the PCAs